note: set `os.environ["AD_PATH"]` or run `export AD_PATH=/path/to/autodiff-opt` before import `ad.grad` (for E2E is wip)

In [2]:
from ad.scalar import *
from ad.grad import *
from mlir.ir import *

define function that has `Scalar`(s) as input and `Scalar`(s) as output

In [3]:
def function(x: Scalar, y: Scalar) -> Scalar:
    return x.log() + x * y - y

call `gen_mlir` to transform python function to mlir module

In [4]:
with Context() as ctx, Location.unknown() as loc:
    primal_module = gen_mlir(ctx, loc, 2, 1, "function", function)
    
print(primal_module)

module {
  func.func @function(%arg0: tensor<1xf32>, %arg1: tensor<1xf32>) -> tensor<1xf32> {
    %0 = "tosa.log"(%arg0) : (tensor<1xf32>) -> tensor<1xf32>
    %1 = "tosa.mul"(%arg0, %arg1) {shift = 0 : i32} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %2 = "tosa.add"(%0, %1) : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %3 = "tosa.sub"(%2, %arg1) : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    return %3 : tensor<1xf32>
  }
}



call `grad_module` to transform primal module to adjoint module as `str`

In [5]:
adjoint_module = grad_module(primal_module)
print(adjoint_module)

module {
  func.func @diff_function(%arg0: tensor<1xf32>, %arg1: tensor<1xf32>, %arg2: tensor<1xf32>) -> (tensor<1xf32>, tensor<1xf32>, tensor<1xf32>) {
    %cst = arith.constant dense<1.000000e+00> : tensor<1xf32>
    %0 = "tosa.log"(%arg0) {requires_grad = 2 : i64} : (tensor<1xf32>) -> tensor<1xf32>
    %1 = "tosa.mul"(%arg0, %arg1) {requires_grad = 5 : i64, shift = 0 : i32} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %2 = "tosa.add"(%0, %1) {requires_grad = 3 : i64} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %3 = "tosa.sub"(%2, %arg1) {requires_grad = 4 : i64} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %4 = "tosa.negate"(%cst) : (tensor<1xf32>) -> tensor<1xf32>
    %5 = "tosa.mul"(%4, %arg2) {shift = 0 : i32} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %6 = "tosa.reciprocal"(%arg0) : (tensor<1xf32>) -> tensor<1xf32>
    %7 = "tosa.mul"(%6, %arg2) {shift = 0 : i32} : (tensor<1xf32>, tensor<1xf32>) -> tensor<1xf32>
    %8 = "tosa.mul"(%arg1, %

call `run_module` to get results

In [6]:
x = 2.
y = 5.
grad = 1.

primal_result = run_module(str(primal_module), "function", x, y)
adjoint_result = run_module(adjoint_module, "diff_function", x, y, grad)

In [7]:
primal_result

5.6931477

In [8]:
adjoint_result

[5.6931477, 5.5, 1.0]